In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words! Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur. We will exclude stopwords and punctuation. In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,behaviour,solitary,twenty,injustice,custom,t,cheerful,hour,begin,dinah,...,seventeen,cause,creep,alloy,modesty,gardener,distress,why,text_sentence,text_source
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


# Trying out BoW

Let's use bag of words using the random forest. 

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9865203761755486

Test set score: 0.8872180451127819


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.8735902255639098


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [16]:
from sklearn.model_selection import GridSearchCV

# Attempt a lasso regression to see if score improves
lr = LogisticRegression(penalty='l1')

# Use GS-CV to see which alpha level is best.
parameters = {
             'C':[0.00001,0.0001,0.001,0.01,0.1,1,10,100,10000,100000,100000000]}

lr_grid = GridSearchCV(lr, parameters, cv=5, verbose=1)

#Fit the logistic regression 
lr_grid.fit(X_train, y_train)

#return best parameters and best score
print('Best parameters:')
print(lr_grid.best_params_)
print('Best Score:')
print(lr_grid.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Best parameters:
{'C': 1}
Best Score:
0.8940438871473354


[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:   31.2s finished


In [18]:
from sklearn.model_selection import cross_val_score
# Implement parameter 
lr = LogisticRegression(C=1, penalty='l1')

# Cross Validate the lr model. 
cross_val_score(lr, X_train, y_train, cv=5)

array([0.885759  , 0.88262911, 0.92476489, 0.8822606 , 0.89324961])

Originally, I thought that using Lasso regression would be a good idea since there were 3062 in the new dataset. I thought that removing some of these columns would be helpful. However, the accuracy of the model was reduced which means that ridge regression is the stronger logistic model since l2 is the default for this model.

In [19]:
# Attempt to find the best parameters for random forest 

rfc = ensemble.RandomForestClassifier()


# Create rfc parameters for GS-CV
param_grid = {'n_estimators': [10, 25, 50, 75, 100, 200, 500, 1000],
             'max_features': [2, 4, 6, 8],
             'max_depth': [2, 4, 6]}

# Set up GS-CV
grid = GridSearchCV(rfc, param_grid, cv=5, verbose=1, n_jobs=-1)

grid.fit(X_train, y_train)

# Show the best parameter and best score for unfiltered
print('Best Parameter for data:', grid.best_params_)
print('\nBest Score for data:', grid.best_score_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  4.9min finished


Best Parameter for data: {'max_depth': 6, 'max_features': 8, 'n_estimators': 10}

Best Score for data: 0.6836990595611285


Random forest isn't worth looking at due to its performance. I don't understand why its performance is lacking compared to the previous model though. 

In [20]:
from sklearn.svm import SVC

svm = SVC()

# new parameters for this model
svc_params = [{'C': [.00001,.0001,.001,.01,.1,1,
                    10,100,1000,10000],
              'gamma': [.00001,.0001,.001,.01,.1,1,10,25,100]}]

# setting up the grid
svc_grid = GridSearchCV(svm, svc_params, cv=5, verbose=1, n_jobs=-1)

#Fit the grid
svc_grid.fit(X_train, y_train)

#return best parameters and best score
print('Best parameters:')
print(svc_grid.best_params_)
print('Best Score:')
print(svc_grid.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 31.8min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 74.8min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed: 77.7min finished


Best parameters:
{'C': 10, 'gamma': 0.01}
Best Score:
0.8952978056426333


In [21]:
# Implement parameters
svm = SVC(C=10, gamma=.01)

# Cross Validate the lr model. 
cross_val_score(svm, X_train, y_train, cv=5)

array([0.88732394, 0.885759  , 0.9137931 , 0.89167975, 0.89795918])

Scores are consistent but I can't seem to get any of my models to break 90%.


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [22]:
# See what I can compare to.
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [23]:
# Load Shakespeare
shakes = gutenberg.raw('shakespeare-caesar.txt')

In [26]:
# Remove chapter names
shakes = re.sub(r'Chapter \d+', '', shakes)
shakes = text_cleaner(shakes)
print(shakes[:100])

Actus Primus. Scoena Prima. Enter Flauius, Murellus, and certaine Commoners ouer the Stage. Flauius.


In [27]:
# Parsed the cleaned novel.
shakes_doc = nlp(shakes)

In [30]:
# Group into sentences.
shakes_sents = [[sent, "Shakespeare"] for sent in shakes_doc.sents]

# Combine sentences
challenge_sentences = pd.DataFrame(alice_sents+shakes_sents)

In [31]:
challenge_sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [32]:
# Set up the bags of words.
shakewords = bag_of_words(shakes_doc)
challenge_common_words = set(alicewords+shakewords)

In [34]:
# Create our data frame with features.
challenge_word_counts = bow_features(challenge_sentences, challenge_common_words)
challenge_word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500


,twenty,t,cin,sawcy,i'th,remain'd,hour,meddle,most,begin,...,little,schoole,cause,creep,driue,bookes,gardener,why,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [35]:
# logistic regression again
Y2 = challenge_word_counts['text_source']
X2 = np.array(challenge_word_counts.drop(['text_sentence','text_source'], 1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, 
                                                    Y2,
                                                    test_size=0.4,
                                                    random_state=0)

In [36]:
# Attempt to find the best parameters for random forest 

rfc = ensemble.RandomForestClassifier()


# Create rfc parameters for GS-CV
param_grid = {'n_estimators': [10, 25, 50, 75, 100, 200, 500, 1000],
             'max_features': [2, 4, 6, 8],
             'max_depth': [2, 4, 6]}

# Set up GS-CV
grid = GridSearchCV(rfc, param_grid, cv=5, verbose=1, n_jobs=-1)

grid.fit(X2_train, y2_train)

# Show the best parameter and best score for unfiltered
print('Best Parameter for data:', grid.best_params_)
print('\nBest Score for data:', grid.best_score_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  3.4min finished


Best Parameter for data: {'max_depth': 6, 'max_features': 6, 'n_estimators': 10}

Best Score for data: 0.6042850896370792


Once again I can't seeem to get my random forest model to perform well.

In [37]:
# Attempt a ridge regression to see if score improves
lr = LogisticRegression(penalty='l1')

# Use GS-CV to see which alpha level is best.
parameters = {
             'C':[0.00001,0.0001,0.001,0.01,0.1,1,10,100,10000,100000,100000000]}

lr_grid = GridSearchCV(lr, parameters, cv=5, verbose=1)

#Fit the logistic regression 
lr_grid.fit(X2_train, y2_train)

#return best parameters and best score
print('Best parameters:')
print(lr_grid.best_params_)
print('Best Score:')
print(lr_grid.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Best parameters:
{'C': 10}
Best Score:
0.9029296020988195


[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:   24.8s finished


In [38]:
# Implement parameter 
lr = LogisticRegression(C=10, penalty='l2')

# Cross Validate the lr model. 
cross_val_score(lr, X_train, y_train, cv=5)

array([0.90453834, 0.88888889, 0.92319749, 0.89795918, 0.89952904])

Consistent results same as before. Let's see how gradient boosting does next.

In [47]:
# Attempt a gradient boosting model to compare.

# Use GS-CV to see which alpha level is best.
clf_parameters = {
             'n_estimators':[100,200,500,1000],
              'max_depth':[2,4,6,8],
              'max_features':[2,4,6,8]
}

clf_grid = GridSearchCV(clf, clf_parameters, cv=5, verbose=1, n_jobs=-1)

#Fit the logistic regression 
clf_grid.fit(X2_train, y2_train)

#return best parameters and best score
print('Best parameters:')
print(clf_grid.best_params_)
print('Best Score:')
print(clf_grid.best_score_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  4.1min finished


Best parameters:
{'max_depth': 8, 'max_features': 6, 'n_estimators': 1000}
Best Score:
0.940096195889812


In [48]:
# Implement parameter 
clf = ensemble.GradientBoostingClassifier(n_estimators=1000, max_depth=8, max_features=6)

# Cross Validate the lr model. 
cross_val_score(clf, X2_train, y2_train, cv=5)

array([0.91721133, 0.95623632, 0.92997812, 0.92778993, 0.9452954 ])

This did way better than the random forest model. This is the chance for gradient boosting to shine since the random forest model did not perform as well. It needed the maximum amount of iterations in order to learn from all the errors from its previous models. It also needed the maximum amount of features and depth in order to increase its accuracy.